<a href="https://colab.research.google.com/github/DavidSenseman/BIO1173/blob/master/Class_02_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---------------------------
**COPYRIGHT NOTICE:** This Jupyterlab Notebook is a Derivative work of [Jeff Heaton](https://github.com/jeffheaton) licensed under the Apache License, Version 2.0 (the "License"); You may not use this file except in compliance with the License. You may obtain a copy of the License at

> [http://www.apache.org/licenses/LICENSE-2.0](http://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

------------------------

# **BIO 1173: Intro Computational Biology**

**Module 2: Machine Learning**

* Instructor: [David Senseman](mailto:David.Senseman@utsa.edu), [Department of Integrative Biology](https://sciences.utsa.edu/integrative-biology/), [UTSA](https://www.utsa.edu/)


### Module 2 Material

* **Part 2.1: Pandas DataFrame Operations**
* Part 2.2: Categorical Values 
* Part 2.3: Grouping, Sorting and Shuffling on Pandas  

### Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [ ]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

### Lesson Setup

Run the next code cell to load necessary packages

In [ ]:
# You MUST run this code cell first
import pandas as pd
import os
import numpy as np
from sklearn import metrics
from scipy.stats import zscore

import os
import shutil
path = '/'
memory = shutil.disk_usage(path)
dirpath = os.getcwd()
print("Your current working directory is : " + dirpath)
print("Disk", memory)

# Part 2.1: Pandas DataFrame Operations

In this lesson we continue our investigation of the software package, Pandas, looking at DataFrame operations that are frequently used in Machine Learning the construction of Deep Neural Networks. 

### Datasets for Class_02_1

In this class we will be using the **_Apple Quality_** dataset for the Examples and the **_Obesity Prediction_** dataset for the Exercises.

### Apple Quality Dataset

[Apple Quality Data Set](https://www.kaggle.com/datasets/nelgiriyewithana/apple-quality)

**Description:**

The Apple Quality dataset contains information about various attributes of a large sample of apples, providing insights into their characteristics. The dataset includes details such as fruit ID, size, weight, sweetness, crunchiness, juiciness, ripeness, acidity, and quality.

**Key Features:**

* **A_id:** Unique identifier for each fruit
* **Size:** Size of the fruit
* **Weight:** Weight of the fruit
* **Sweetness:** Degree of sweetness of the fruit
* **Crunchiness:** Texture indicating the crunchiness of the fruit
* **Juiciness:** Level of juiciness of the fruit
* **Ripeness:** Stage of ripeness of the fruit
* **Acidity:** Acidity level of the fruit
* **Quality:** Overall quality of the fruit

### Example 1: Read data file into a Pandas DataFrame

The cell below use the Pandas `read_csv()` function to read the `apple_quality.csv` file that is located on the course HTTPS server using the following code chunk: 
~~~text
> apOrigDF = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/apple_quality.csv",
    na_values=['NA','?']))
~~~

The function`read_csv()` is an important Pandas function for reading CSV files. In the cell below, the `read_csv()` function takes 2 arguments. The first argument,

>`"https://biologicslab.co/BIO1173/data/apple_quality.csv",`

is a string that provides the filepath and filename of the datafile to be read. 

The second argument: 

> `na_values=['NA','?']`

is used to convert any missing data points in the file (`?`) into the value, `NaN` which stands for Not-a-Number.

As the file is read, Pandas creates a Pandas DataFrame called `apOrigDF` to hold the information. The Pandas `df.copy()` method is then used to create a copy of the original data in DataFrame called `apDF`. The `apDF` DataFrame will be used for most of the examples below.

After reading the datafile into a DataFrame, it is always a good idea to use the `display()` function a print out a specified number of rows and columns to make sure the data was read correctly. In the cell below, the `pd.set_option()` function is used twice, once to specify the number of columns and once again to specify the number of rows.  Many datasets have both a large number of rows and columns which can not be printed out easily to your notebook.

In [ ]:
# Example 1: Read the datafile and create a Pandas DataFrame

# Read the datafile 
apOrigDF = pd.read_csv(
    "https://biologicslab.co/BIO1173/data/apple_quality.csv",
    na_values=['NA','?'])

# Create a copy
apDF = apOrigDF.copy() 

# Set the display for 12 rows and 6 columns
pd.set_option('display.max_rows', 12)
pd.set_option('display.max_columns', 6)

# Display the DataFrame
display(apDF)

If your code is correct you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_02_1_Exm1.png)

From the output, we can see that `apDF` has 4000 rows and 9 columns. Only 6 of the 9 columns are printed out. The missing columns are represented by the `...`.

In a Pandas DataFrame, each row is an _observation_, in this case a single piece of fruit (apple). The 9 columns record the various **_Key Features_**, or factors, that were measured for each apple.

### Obesity Prediction Dataset

[Obesity Prediction Dataset](https://www.kaggle.com/datasets/mrsimple07/obesity-prediction)

**Description**

The Obesity Prediction dataset provides comprehensive information on individuals' demographic characteristics, physical attributes, and lifestyle habits, aiming to facilitate the analysis and prediction of obesity prevalence. It includes key variables such as age, gender, height, weight, body mass index (BMI), physical activity level, and obesity category. 

* **Age:** The age of the individual, expressed in years.
* **Gender:** The gender of the individual, categorized as male or female.
* **Height:** The height of the individual, typically measured in centimeters or inches.
* **Weight:** The weight of the individual, typically measured in kilograms or pounds.
* **BMI:** A calculated metric derived from the individual's weight and height
* **PhysicalActivityLevel:** This variable quantifies the individual's level of physical activity
* **ObesityCategory:** Categorization of individuals based on their BMI into different obesity categories

### **Exercise 1: Read data file into a Pandas DataFrame**

In the cell below, use the Pandas `read_csv()` function to read the `obesity_prediction.csv` file that is located in your `../Datasets/` folder. 

As the file is read, have Pandas create a Pandas DataFrame called `obOrigDF` to hold the information. After reading the datafile into the DataFrame, use the Pandas `df.copy()` method to create a copy of the original data called `obDF`. This will be the dataframe that you will use in most of the exercises below. Finally, use the `display()` function to print out 7 columns and 8 rows of the `obDF` DataFrame.


In [ ]:
# Insert your code for Exercise 1 here



If your code is correct you should see the following table.

![___](https://biologicslab.co/BIO1173/images/class_02_1_ObesityDF.png)

As you can see, the `obDF` DataFrame contains information about 1000 patients/subjects (1 row per patient) with 7 measurements for each patient. 

## Dealing with Outliers

Outliers are values that are unusually high or low. We typically consider outliers to be a value that is several standard deviations from the mean. Sometimes outliers are simply errors; this is a result of [observation error](https://en.wikipedia.org/wiki/Observational_error). Outliers can also be truly large or small values that may be difficult to address. The following **_function_** can remove such values. The code in the function below will drop any row of data that contains a selected value that is a specified number of standard deviations above, or below, the mean.   

In [ ]:
# FUNCTION TO REMOVE OUTLIERS

# Remove all rows where the specified column is +/- sd standard deviations
def remove_outliers(df, name, sd):
    drop_rows = df.index[(np.abs(df[name] - df[name].mean())
                          >= (sd * df[name].std()))]
    df.drop(drop_rows, axis=0, inplace=True)

--------------------------------------

### **FUNCTIONS**

A Python **_function_** is a reusable block of code that performs a specific task. It is defined using the `def` keyword followed by the _function name_ and parentheses. Functions can take input values called arguments or parameters, which are defined inside the parentheses. These parameters can be used within the function's code block to perform computations or operations.

Functions in Python are organized, modular, and help in code reusability. They promote code readability and maintainability by dividing complex logic into smaller, manageable units. They also allow programmers to encapsulate functionality and use it repeatedly throughout the program, improving efficiency and reducing duplication.

Functions can have a return statement that provides the computed result or value back to the caller. This returned value can be stored in a variable or used directly. Functions can also be called within other functions, allowing for nested function calls. Overall, Python functions are a fundamental building block in programming that enables code organization, reusability, and abstraction.

-----------------------------------------

### Example 2: Remove Rows with outliers

The cell below uses the function `remove_outliers()`, created above, to remove outliers in the Apple Quality dataset. In order to use this function, you need to decide which feature (i.e. `column`) that you want to test for outliers. In this example, the focus is on the `Acidity` measurement. 

Before we can apply the function to the DataFrame, we must create a "feature vector" to take care of any records in which the acidity value is missing (i.e. NaN). A feature vector usually consists of numeric values, in this case the statistical [median](https://en.wikipedia.org/wiki/Median) of the acidity level in all of the apples in the sample. This is done using `median()` as follows:

> `med = apDF['Acidity'].median()`
>

Notice that we are using square bracket indexing to select **only** the column labeled `'Acidity'`. We then use the median acidity value to "fill in" any blank spaces with the following code line:

> `apDF['Acidity'] = apDF['Acidity'].fillna(med)`
>

It should be noted that replacing any missing value by the column's median value, keeps the column's median value the same. 

Having filled in any missing acidity values, we can now use our function to remove outliers: 

> `remove_outliers(apDF,'Acidity',2)`
>

The number `2` specifies the number of [standard deviations](https://en.wikipedia.org/wiki/Standard_deviation) for our upper and lower boundary. The function will remove any apple (i.e. row) from dataframe in which the acidity value that is greater than `2` standard deviations (sd) above, or below, the mean. 

Finally, to see if the function worked, we print out the number of rows in the DataFrame before, and then after, applying the function.


In [ ]:
# Example 1: Use remove_outlier() function with Apple Quality dataset

# create feature vector
med = apDF['Acidity'].median()   # calculate the median value for acidity
apDF['Acidity'] = apDF['Acidity'].fillna(med)  # replace an NaN with median 

# Print the number of rows before applying the function
print("Length before Acidity outliers dropped: {}".format(len(apDF)))

# Apply the funtion to the Acidity column
remove_outliers(apDF,'Acidity',2)

# Print the number of rows before applying the function
print("Length after Acidity outliers dropped: {}".format(len(apDF)))

# Set the display for 5 rows and 8 columns
pd.set_option('display.max_rows', 5)
pd.set_option('display.max_columns', 8)

# Display the dataframe
display(apDF)

If your code is correct you should see the following table:

![__](https://biologicslab.co/BIO1173/images/class_02_1_Drop1.png)

Notice that after applying the `remove_outliers()` function, there are now 184 fewer rows (apples) in the `apDF` DataFrame. If you have had statistics, this is almost exactly what you would expect! According to the [**_68%-95%-99.7% Rule_**](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule), 95% of a normally distributed variable (in this case `acidity`) should be within 2 standard deviations of the mean. If you multiple 4000 apples X 0.95, you get 3,800. In other words, you should expect about 200 of the 4,000 apples would have an acidity that was either 2 standard deviations too high, or too low which is why they were removed from the DataFrame. This is almost exactly what we got. 

### **Exercise 2:  Remove Rows with outliers**

In the cell below, write the Python code to remove outliers from the `obDF` DataFrame using the `remove_outliers()` function created above. Focus on the characteristic called `BMI` that stands for Body Mass Index. Only discard patients (rows) that have a `BMI` that is more than `3` standard deviations (sd) from the mean. Make sure to fill in any missing `BMI` values with the median value for the `BMI` column. 

When you are done, print out `7` columns and `5` rows of the `obDF` DataFrame.

In [ ]:
# Insert your code for Exercise 2 here



If your code is correct you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_02_1_Drop2.png)

After applying the `remove_outliers()` function with `sd=3`, you should have removed only 4 patients, leaving 996 patients (rows) in the `obDF` dataframe.

Again, using the [**_68%-95%-99.7% Rule_**](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule), 99.7% of a normally distributed variable (in this case `BMI`) should be within 3 standard deviations of the mean. If you multiple 1000 patients X 0.997, you get 997. Again, the number of rows that were removed (4) is almost exactly what would be predicted.

## Concatenating Rows and Columns
Python can concatenate rows and columns together to form new data frames. In Pandas, **_concatenation_** refers to the process of combining and merging two or more DataFrames or series along a particular axis to create a new data structure. It allows us to stack or join DataFrames/series vertically or horizontally.

Pandas provides the `concat()` function to perform concatenation. By default, concatenation is done vertically along `axis 0`, resulting in a new DataFrame/series with rows appended. However, you can specify `axis=1` to concatenate horizontally, merging columns. The program does this by concatenating two columns together.

In the example below concatenation will be done along `axis 1` so the columns end up side-by-side (horizontally).

### Example 3: Concatenate Pandas columns

The cell below shows the Python code for creating a new DataFrame called `catDF` by extracting the columns `Sweetness` and `Quality` from the `apDF` DataFrame. The code then uses the Pandas `pd.concat()` function to combine these two columns together, side-by-side, into a new DataFrame called `catDF`.

In [ ]:
# Example 3: Concatenate Pandas columns

# Create a new DataFrame from Sweetness and Quality

# Extract specific columns from apDF
col_sweet = apDF['Sweetness']
col_qual = apDF['Quality']

# Use Pandas concat() function to add them side-by-side
catDF = pd.concat([col_sweet, col_qual], axis=1)

# Set the display for 8 rows and all columns
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 0)

# Display new DataFrame
display(catDF)

If your code is correct you should see the following table:

![__](https://biologicslab.co/BIO1173/images/class_02_1_Con1.png)


### **Exercise 3: Concatenate Pandas columns** 

In the cell below write the Python code to extract the columns `ObesityCategory` and `BMI` from the `obDF` DataFrame. Use the Pandas `concat()` function to combine these two columns vertically to create a new DataFrame called `resDF` (for _result_ dataframe). Use the `display()` function to print out all of the columns and 8 rows.

In [ ]:
# Insert your code for Exercise 3 here



If your code is correct you should see the following table:

![__](https://biologicslab.co/BIO1173/images/class_02_1_Con2.png)


### Example 4: Concatenate Pandas rows

The **concat** function can also concatenate rows together.  The code in the cell below concatenates the first three rows and the last thress rows of the Apple Quality dataset.

The code uses square bracket indexing to specify which rows to include in a new DataFrame called `catDF`. The code:

> `[apDF[0:3]`
>
specifies all of the rows, from the first row, index `0`, up to but **not** including index `3` which is the 4th row.

The code:

> `[apDF[-3: ]`
>
specifies all of the rows starting at the 3rd row from the end, up to and including the last row.

The code:

> `axis=0`
>

Specifies the operation should be done on the rows not the columns.


In [ ]:
# Example 4: Create a new DataFrame from first 2 rows and last 2 rows

# Use Pandas concat() function
catDF = pd.concat([apDF[0:3],apDF[-3: ]], axis=0)

# Set the display for all rows and 6 columns
pd.set_option('display.max_rows', 0)
pd.set_option('display.max_columns', 6)

# Display new dataframe
display(catDF)

If your code is correct you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_02_1_Exm4.png)


### **Exercise 4: Concatenate Pandas rows**

In the cell below, use the Pandas `concat()` function to concatenate the first two and last two rows of the Obesity Prevention dataset to create a new DataFrame called `resDF`. Display _all_ of the columns, and _all_ of the rows, in `resDF`. 

In [ ]:
# Insert your code for Exercise 4 here



If your code is correct you should see the following table:

![___](https://biologicslab.co/BIO1173/images/class_02_1_Res2.png)


## Training and Validation

We must evaluate a machine learning model based on its ability to predict values that it has never seen before. Because of this, we often divide the training data into a validation and training set. The machine learning model will learn from the training data but ultimately be evaluated based on the validation data.

* **Training Data** - **In Sample Data** - The data that the neural network used to train. 
* **Validation Data** - **Out of Sample Data** - The data that the machine learning model is evaluated upon after it is fit to the training data.

There are two effective means of dealing with training and validation data:

* **Training/Validation Split** - The program splits the data according to some ratio between a training and validation (hold-out) set. Typical rates are 80% training and 20% validation.
* **K-Fold Cross Validation** - The program splits the data into several folds and models. Because the program creates the same number of models as folds, the program can generate out-of-sample predictions for the entire dataset.

The code below splits the data into a training and validation set. The training set uses 80% of the data, and the validation set uses 20%. Figure 2.TRN-VAL shows how we train a model on 80% of the data and then validated against the remaining 20%.

**Figure 2.TRN-VAL: Training and Validation**
![Training and Validation](https://biologicslab.co/BIO1173/images/class_1_train_val.png "Training and Validation")


### Example 5: Divide data into Training and Validation sets

The code in the cell below begins by shuffling the `apDF` dataset using Pandas `reindex()` method combined with the Numpy `random.permutation()` function. Once the data has been shuffled, the code splits the data in the Apple Quality dataset into a _Training_ DataFrame called `ap_TrainDf` and a _Validation_ DataFrame called `ap_validationDF`. 

To do this, the code first creates a _boolean mask_ called `mask` (see the next cell for a discussion of boolean masks) using this line of code:

> `mask = np.random.rand(len(apDF)) < 0.8`
>

The code creates a `mask` that is a Numpy array containing the values either `True` or `False`. The number of `True` and `False` values is exactly the same as the number data points in the complete dataset. Approximately 80% of the values in the `mask` are `True` and the remaining are the value `False`.  

To figure out the exact number of `True` values in the `mask`, we take advantage of the fact that a `True` is equal to the number `1` while a `False` is equal to the number `0`. Therefore, to compute the number of `True` values, we can simply compute the `sum` for the `mask` as shown by this line of code:

> `print(f"The number of True values in the mask: {sum(mask)}")`
>

To create the validation DataFrame, the following line of code is used:

>  `apValidationDF = pd.DataFrame(apDF[~mask])`
>

The character `~` before the `mask` is called a _tilde_. The ~ (tilde) operator in Python is used as a unary operator to perform the bitwise inversion operation on integers. It flips the bits of an integer, changing 0s to 1s and 1s to 0s. In other words, `[~mask]` means to flip every `True` to a `False` and every `False` to a `True`. Since the `mask` is inverted, it now "selects" the roughly 20% of the complete dataset that was **_not_** placed in the Training Dataframe, to be included in the Validation Dataframe.


In [ ]:
# Example 5: Split apDF into a training DF and a validation DF

# Make sure the training DF contains 80% of the data
# And the validation DF contains the remaining 20%

# Usually a good idea to shuffle
apDF = apDF.reindex(np.random.permutation(apDF.index)) 

# Create a boolean mask that is 80% the length of the entire dataset
mask = np.random.rand(len(apDF)) < 0.80 

# Apply the mask to the whole data to produce the training DF
apTrainDF = pd.DataFrame(apDF[mask])

# Using the inverse of the mask to generate the validation DF
apValidationDF = pd.DataFrame(apDF[~mask])

# Print out the lengths of training and validation DF's
print(f"The original Apple DF: {len(apDF)}")
print(f"The number of True values in the mask: {sum(mask)}")
print(f"Apple Training DF: {len(apTrainDF)}")
print(f"Apple Validation DF: {len(apValidationDF)}")

If your code is correct you should see the following output:
~~~text
The original Apple DF: 3816
The number of True values in the mask: 3071
Apple Training DF: 3071
Apple Validation DF: 745
~~~
Since the generation the boolean mask is a _random process_, the number of records in the `Training DF` and the `Validation DF` will vary slightly each time the code is run. However, the number `True` values in the `mask` will _always_ be the same the length of `Training DF` and the sum of `Training DF` and the `Validation DF` will _always_ be equal to the same length as the original `Apple DF`. 

------------------------------------------------------------
### **MASKS**

A mask in Pandas refers to a boolean series or a boolean array that is used to filter or select specific rows from a DataFrame based on certain conditions. It acts as a filter to determine which rows should be included or excluded in subsequent operations.

The mask is a result of applying a logical condition to a dataframe, resulting in a series or an array where each element holds a boolean value indicating whether the condition is `True` or `False` for that particular row. The `mask` can be used to extract only the rows that satisfy the given condition.

For example, consider a dataframe named `DF` with columns `A` and `B`. To create a mask using a condition (e.g., select rows where column `A` is greater than `5`), you can use the following syntax:

>  `mask = DF['A'] > 5`
>
This will generate a _boolean series_ in mask, where each element represents if the corresponding row in column `A` satisfies the condition (`True`) or not (`False`).

To apply this mask and obtain the filtered dataframe, `filteredDF`, containing only the rows where column `A` is greater than `5`, you can use the following syntax:

>  `filteredDF = DF[mask]`
>
The resulting `filteredDF` will contain only the rows where the condition in the mask is `True`.

Masks are particularly useful for conditional filtering operations and allow for easy extraction of subsets of data based on specific criteria.

-----------------------------------------------------


### **Exercise 5: Divide data into Training and Validation sets** 

In the cell below write the Python code to split the Obesity Prediction data into a training set called `obTrainDF` with 80% of the data and a validation set called `obValidationDF` with the remaining 20%. Make sure to shuffle the data before you split the data. Finally, print out the length of original dataset (i.e. `obDF`), the number of `True` values in the `mask`, as well as the lengths of the training and validation sets.  

In [ ]:
# Insert your code for Exercise 5 here



If your code is correct you should see something similar, but not necessary identical to the following output:

~~~text
The original Obesity Predicti0n DF: 996
The number of True values in the mask: 801
Obesity Training DF: 801
Validation DF: 195
~~~

The reason for the variability in the output is due to the random process used to generate the boolean `mask`.

## Converting a DataFrame to a Matrix

There are several reasons why you might want to convert a Pandas DataFrame into a numerical array. For example:

* **Mathematical Operations:** If you need to perform mathematical operations on the data, converting the DataFrame to a numerical array can be advantageous. Numpy arrays allow for efficient numerical computations and provide a wide range of mathematical functions.

* **Integration with Machine Learning Libraries:** Many machine learning libraries, such as Scikit-learn, expect input in the form of numerical arrays. Converting your DataFrame to a numerical array allows you to seamlessly integrate and use these libraries for tasks such as classification, regression, or clustering.

* **Memory Efficiency:** Numpy arrays are more memory-efficient compared to Pandas DataFrames. If you have a large dataset and memory consumption is a concern, converting the DataFrame to a numerical array can help reduce memory usage.

* **Compatibility with Statistical Packages:** Statistical packages like `SciPy` or `Statsmodels` often work more efficiently with numerical arrays. Converting your DataFrame to a numerical array can enhance compatibility and facilitate statistical analysis or hypothesis testing.

* **Neural networks:** Neural networks do not directly operate on Python DataFrames.  A neural network requires a numeric matrix.  

### Example 6: Convert a DataFrame into matrix

The Python code in the cell below uses the Pandas `.values()` method to convert values in a DataFrame into a matrix. For this example, we are using the original Apple Quality dataset that was stored in the `apOrigDF` DataFrame to create an **_array_** called `apAR`. The suffix `AR` is used to indicate the variable is an array. 


In [ ]:
# Example 6: Convert dataframe into a matrix

# Use the Pandas .values method
apAR = apOrigDF.values

# Print out the new array
apAR

If your code is correct you should see the following output:
~~~text
array([[0, -3.970048523, -2.512336381, ..., 0.329839797, -0.491590483,
        'good'],
       [1, -1.195217191, -2.839256528, ..., 0.867530082, -0.722809367,
        'good'],
       [2, -0.292023862, -1.351281995, ..., -0.038033328, 2.621636473,
        'bad'],
       ...,
       [3997, -2.634515299, -2.13824672, ..., 4.763859177, -1.334611391,
        'bad'],
       [3998, -4.008003744, -1.779337107, ..., 0.214488384, -2.229719806,
        'good'],
       [3999, 0.27853965, -1.715505028, ..., -0.77657147, 1.599796456,
        'good']], dtype=object)
~~~

### **Exercise 6: Convert a DataFrame into matrix**

In the cell below use the Pandas `df.values()` method to convert values in the `obOrigDF` DataFrame into a matrix called `obAR`. Then print out the values of `obAR`.


In [ ]:
# Insert your code for Exercise 6 here



If your code is correct you should see the following output:

~~~text
array([[56, 'Male', 173.5752624383722, ..., 23.89178262396797, 4,
        'Normal weight'],
       [69, 'Male', 164.1273058223382, ..., 33.39520945079775, 2,
        'Obese'],
       [46, 'Female', 168.0722021276139, ..., 25.81773745564312, 4,
        'Overweight'],
       ...,
       [49, 'Female', 156.57095601730393, ..., 32.14603570597436, 1,
        'Obese'],
       [64, 'Male', 164.1922221149603, ..., 21.505964950969545, 4,
        'Normal weight'],
       [66, 'Female', 178.5371304568283, ..., 23.51716758335692, 1,
        'Normal weight']], dtype=object)
~~~

### Example 7: Convert a subset of columns to a matrix

You might wish only to convert some of the columns in a DataFrame into a matrix, and leave out the other columns. As a first step, the code in the cell below just shows how to print out just the names of the columns in the DataFrame.

In [ ]:
# Example 7A List column names

# use the list function
list(apOrigDF.columns.values)

You should see the following output:
~~~text
['A_id',
 'Size',
 'Weight',
 'Sweetness',
 'Crunchiness',
 'Juiciness',
 'Ripeness',
 'Acidity',
 'Quality']
~~~
Once you know the names of the columns, you can select which columns to include in the matrix (array). In the cell below, the columns `A_id` and `Quality` will _not_ included in the matrix.

In [ ]:
# Example 7B: Select specific columns for a matrix

# Select the columns to include in the matrix
selectAR = apOrigDF[['Size', 'Weight', 'Sweetness', 'Crunchiness', 'Juiciness',
       'Ripeness']].values

# Print new array
selectAR

You should see the following output:
~~~text
array([[-3.97004852, -2.51233638,  5.34632961, -1.01200871,  1.84490036,
         0.3298398 ],
       [-1.19521719, -2.83925653,  3.66405876,  1.58823231,  0.8532858 ,
         0.86753008],
       [-0.29202386, -1.35128199, -1.73842916, -0.34261593,  2.83863551,
        -0.03803333],
       ...,
       [-2.6345153 , -2.13824672, -2.44046129,  0.65722289,  2.19970859,
         4.76385918],
       [-4.00800374, -1.77933711,  2.36639697, -0.20032937,  2.16143512,
         0.21448838],
       [ 0.27853965, -1.71550503,  0.12121725, -1.15407476,  1.2666774 ,
        -0.77657147]])
~~~

### **Exercise 7A: Convert a subset of columns to a matrix**

In the cell below print out the column names in the DataFrame, `obOrigDF`.

In [ ]:
# Example 7A List column names



If your code is correct you should see the following output:

~~~text
['Age',
 'Gender',
 'Height',
 'Weight',
 'BMI',
 'PhysicalActivityLevel',
 'ObesityCategory']
~~~

Now that you know the names of the columns, select only the columns `Age`, `Gender`, `Height` and `Weight` to include in a new array called `selectAR`. Do _not_ include the columns `BMI`, `PhysicalActivityLevel` or `ObesityCategory` in the matrix.

In [ ]:
# Example 7B: Select specific columns for a matrix

# Select the columns to include in the matrix
selectAR = obOrigDF[['Age', 'Gender', 'Height', 'Weight']].values

# Print new array
selectAR

If your code is correct you should see the following output:

~~~text
array([[56, 'Male', 173.5752624383722, 71.98205082003972],
       [69, 'Male', 164.1273058223382, 89.95925553264384],
       [46, 'Female', 168.0722021276139, 72.93062926527617],
       ...,
       [49, 'Female', 156.57095601730393, 78.80428436775063],
       [64, 'Male', 164.1922221149603, 57.97811543905408],
       [66, 'Female', 178.5371304568283, 74.96216387889635]], dtype=object)
~~~

## Saving a DataFrame to CSV

You might want to convert a Pandas DataFrame into a CSV file in various situations, including:

* **Data Storage:** CSV (Comma-Separated Values) is a commonly used file format for storing tabular data. If you need to store your DataFrame as a standalone file, converting it to a CSV format allows for easy sharing, portability, and compatibility with other software.

* **Data Exchange:** CSV is widely recognized and supported by numerous applications and programming languages. When you want to exchange data with other systems, convert your DataFrame to a CSV file to ensure seamless interoperability and enable the recipients to access and process the data without requiring Pandas or a specific library.

* **Data Analysis:** Some statistical or data analysis software prefer CSV files as input. By converting your DataFrame to a CSV file, you can leverage these external tools or libraries for advanced analysis, visualization, or modeling.

* **Database Import:** Many databases and data storage systems accept CSV files as a means of data insertion. By converting your DataFrame to a CSV file, you can easily import the data into a database, making it more manageable, searchable, and suitable for long-term storage.

* **Data Backup:** Creating a CSV file from your DataFrame acts as a backup option, ensuring that your data remains accessible even if something happens to the original DataFrame or its environment. This can serve as a contingency plan or for version control purposes.

Converting a Pandas DataFrame into a CSV file allows you to save, exchange, analyze, and backup your data efficiently, ensuring flexibility, compatibility, and ease of use across various applications and systems.


### Example 8: Saving a DataFrame to CSV

The code in the cell below saves the DataFrame `apOrigDF` to a CVS file called `AppleQualityCSV.csv`. 

In [ ]:
# Example 8A: Save a DataFrame to CSV

# Specify the path
path = "."

# Specify the file path and filename
filename_write = os.path.join(path, "AppleQualityCSV.csv")

# Shuffle the data before saving
apOrigDF = apOrigDF.reindex(np.random.permutation(apOrigDF.index))

# Specify index = false to not write row numbers
apOrigDF.to_csv(filename_write, index=False) 

You should now see the file `AppleQualityCSV.csv` in your file browser panel.

The code in the next cell reads the new file `AppleQualityCSV.csv` back into a new DataFrame called simply `df`. This is done to illustrate what data looks like when saved to a CSV file.

In [ ]:
# Example 8B: Read the new CSV file

# Use read_csv() function to read data and create dataframe
df = pd.read_csv("./AppleQualityCSV.csv", na_values=['NA','?'])


# Set the display for 12 rows and 6 columns
pd.set_option('display.max_rows', 12)
pd.set_option('display.max_columns', 6)

# Display 6 columns and 12 rows of the dataframe
display(df)


Notice that even though we had shuffled the data before writing it the CSV file in Example 8A, the index numbers at the left have been restored. The data is still scrambled (compared to the original CSV datafile) but Pandas writes the dataframe using sequential index numbers. 

As you will see shortly, this is _not_ what will happen below when the data is written and then read back into memory using Pickel. 

### **Exercise 8: Saving a DataFrame to CSV**

In the cell below, save the DataFrame `obOrigDF` to a CVS file called `ObesityPredictionCSV.csv`. 

In [ ]:
# Insert your code for Exercise 8 here



You should now see the file `ObesityPredictionCSV.csv` in your file browser panel. There is no need to read your new CSV file.

## Saving a DataFrame to Pickle

A variety of software programs can use text files stored as CSV. However, they take longer to generate and can sometimes lose small amounts of precision in the conversion. Generally, you will output to CSV because it is very compatible, even outside of Python.

Another file format is [Pickle](https://docs.python.org/3/library/pickle.html). The code below stores the DataFrame to the Pickle file format. Pickle stores data in the **_exact binary representation_** used by Python. The benefit is that there is no loss of data going to CSV format. The disadvantage is that generally, only Python programs can read Pickle files.

### Example 9: Saving a DataFrame to Pickle

The code in the cell below saves the DataFrame `apOrigDF` to a Pickel file called `AppleQualityPickel.pkl`. Before you can either read or save a Pickel file, you need to import the `pickel` module. 

The line of code:

>   `with open(filename_write,"wb") as fp:`
>
opens a _file pointer_ `fp`. The argument `"wb"` tells Python that you are going to Write Binary data to the file you just opened. 

After writing to the file, the command `fp.close()` is used to _close the file_. Closing the file is essential to release system resources and ensure that any changes or data in the buffer are properly written to the disk. It is recommended to close the file as soon as you have finished working with it, to maintain good programming practices and avoid potential issues with file handling.

In [ ]:
# Example 9: Save a dataframe to pickle

# Import the pickle module
import pickle

# Specify the path
path = "."

# Specify the file path and filename
filename_write = os.path.join(path, "AppleQualityPickel.pkl")

# Shuffle the data before saving
apOrigDF = apOrigDF.reindex(np.random.permutation(apOrigDF.index))

# Write out the dataframe to a pickel file
with open(filename_write,"wb") as fp:
    pickle.dump(apOrigDF, fp)

# Close the file after writing to it
fp.close()

You should now see the file `AppleQualityPickel.pkl` in your file browser panel.

### **Exercise 9: Saving a DataFrame to Pickle**

In the cell below, save the DataFrame `obOriginalDF` to a Pickel file called `ObesityPredictionPickel.pkl`. Make sure to shuffle the data before writing to the Pickel file and to close the file after you are done writing to it.

In [ ]:
# Insert your code for Exercise 9 here



You should now see the file `ObesityPredictionPickel.pkl` in your file browser panel.

## Loading a Pickel File into Memory

You might want to load a Pickle file into memory in the following scenarios:

* **Preserving Data Structure:** When you have a complex data structure, such as a nested dictionary, list of dictionaries, or custom objects, Pickle allows you to serialize and deserialize the data, preserving its original structure. Loading a Pickle file back into memory ensures that you can restore the data structure exactly as it was when it was saved.

* **Efficient Storage and Retrieval:** Pickle provides a convenient way to store large amounts of data in a compact binary format. If you have large datasets or complex objects that you need to save and retrieve efficiently, using Pickle files can offer significant advantages over other formats like CSV or JSON.

* **Python Object Serialization:** Pickle is a native Python module, specifically designed for serializing Python objects. If you have Python-specific objects or data structures that you want to save and restore without losing any information or functionality, loading a Pickle file allows you to recreate the objects exactly as they were when they were pickled.

* **Data Persistence:** Pickle can be used for persistent storage of data, allowing you to save and load data between program executions. By loading a Pickle file into memory, you can access and utilize the previously saved data, eliminating the need to recreate or recalculate it each time the program runs.

* **Easy Interoperability:** Pickle files can be shared and used across different Python environments and versions, ensuring compatibility and easy transfer of data between systems. Loading a Pickle file is a quick and efficient way to import saved data in a format that can be readily understood and processed by any environment that supports Pickle.


### Example 10: Load a Pickel file into memory

Loading the pickle file back into memory is accomplished by the following lines of code.  Here we are using Pickel's `load()` function to read the file. 

In [ ]:
# Example 10: Load a pickel file into memory

# Specify the path
path = "."

# Specify the file path and filename
filename_read = os.path.join(path, "AppleQualityPickel.pkl")

# Open up a file pointer fp
with open(filename_write,"rb") as fp:
    apPickelDF = pickle.load(fp)

# Close the file after reading it
fp.close()

# Set the display for 8 rows and 7 columns
pd.set_option('display.max_rows', 8)
pd.set_option('display.max_columns', 7)

# Display the new dataframe
display(apPickelDF)

Notice that the index numbers at the left side are still jumbled from the previous shuffle. 

-----------------------------------

## **Pickel Module**

Python's **_pickle module_** provides functionality for serializing and deserializing Python objects. The `pickle.load()` method is used to deserialize and load a serialized object from a Pickle file or a file-like object.

Here are key points about the `pickle.load()` method:

* **Purpose:** The `pickle.load()` method is used to deserialize and load a Pickle object that was previously serialized with pickle.dump() or pickle.dumps().

* **Usage:** To use `pickle.load()`, you first need to open a file in binary mode using the `open()` function and pass the file object as an argument to `pickle.load()`. Alternatively, you can use a file-like object that supports the necessary read operations.

* **Deserialization:** `pickle.load()` reads the serialized object from the file or file-like object and reconstructs the original Python object in memory. It restores the object's state, including its attributes, methods, and other details.

* **Unpickling Security:** It's crucial to note that loading Pickle data can have security implications. Untrusted or malicious Pickle files can execute arbitrary code when loaded using `pickle.load()`. Only load Pickle data from trusted sources to prevent security risks.

* **Handling Different Pickle Formats:** The `pickle.load()` method can handle different Pickle format versions. It detects the Pickle format automatically and loads the object accordingly.

* **Closing the File:** After loading the Pickle object successfully, remember to close the file using the close() method of the file object or by utilizing a with statement to ensure proper resource management.

Using the `pickle.load()` method provides a convenient way to deserialize and load serialized Python objects. However, be cautious and load Pickle data only from trusted sources to avoid potential security vulnerabilities.

------------------------------------

### **Exercise 10: Load a Pickel file into memory**

In the cell below, load the pickle file `ObesityPredictionPickel.pkl` back into memory  using Pickel's `load()` function, creating a DataFrame called `obPickelDF`. Display 7 columns and 8 rows of `obPickelDF`.

In [ ]:
# Insert your code for Exercise 10 here



If your code is correct you should see something similar the following output:

![___](https://biologicslab.co/BIO1173/images/class_02_1_Pickel1.png).

Notice that the index numbers at the left side are still jumbled from the previous shuffle. 

## **Lesson Turn-in**

When you have completed all of the code cells, and run them in sequential order (the last code cell should be number 26), use the **File --> Print.. --> Save to PDF** to generate a PDF of your JupyterLab notebook. Save your PDF as `Class_02_1.lastname.pdf` where _lastname_ is your last name, and upload the file to Canvas.